In [1]:
import json
import os
from glob import glob
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb

from IPython.display import display, HTML

In [2]:
api = wandb.Api()

In [3]:
# The W&B API also provides a way for you to query across runs in a project with api.runs(). The most common use case is exporting runs data for custom analysis.  The query interface is the same as the one MongoDB uses.
# wandb.init("analysis", mode="offline")
def download_test_runs(overwrite=False):
    runs = api.runs("ajayj/nerf-nl", {"$and": [{"config.render_only": True, "config.render_test": True, "state": "finished"}]},
                    order='-created_at')
    print("Found %i" % len(runs))

    # Get most recent run of each
    runs_by_expname = {}
    for run in runs:
        expname = run.config["expname"]
        if expname not in runs_by_expname:
            runs_by_expname[expname] = run
    print("Filtered to %i most recent and unique" % len(runs_by_expname))

    # Download metrics
    for run in runs_by_expname.values():
        #metrics_path = f'logs/{run.config["expname"]}/renderonly_test_*/test_metrics.json'
        files = run.files()
        for file in files:
            if file.name.startswith(f'logs/{run.config["expname"]}/renderonly_test_') and file.name.endswith('/test_metrics.json'):
                if not overwrite and os.path.exists(file.name):
                    continue
                print('Downloading', file.name)

                try:
                    file.download(root='.', replace=True)
                except Exception as e:
                    print('download error', file.name, e)

    return runs_by_expname

In [4]:
runs_by_expname = download_test_runs()

Found 97
Filtered to 47 most recent and unique


In [5]:
config_blacklist = [
    'configs/257_blender_materials_8views_ctr.txt',
]

In [6]:
# Load metrics into a dataframe
# print("Skipping", set(glob('logs/*/renderonly_test_*/test_metrics.json')) - set(glob('logs/*/renderonly_test_199999/test_metrics.json')))

metrics_list = []
for metrics_path in glob('logs/*/renderonly_test_*/test_metrics.json'):
    with open(metrics_path, 'r') as metrics_f:
        metrics_and_args = json.load(metrics_f)
        
        # extract iteration count
        # metrics_and_args['test_metrics_path'] = metrics_path
        m = re.match(r'logs/.+/renderonly_test_([0-9]+)/test_metrics.json', metrics_path)
        assert m
        metrics_and_args['train_iters'] = int(m.group(1))
        # extract training view count
        metrics_and_args['views'] = metrics_and_args["args"]['max_train_views']
        metrics_and_args['views'] = metrics_and_args['views'] if metrics_and_args['views'] >= 0 else 100

        if metrics_and_args['args']['consistency_loss'] != 'none':
            metrics_and_args['method'] = 'SCaRF (ours)'
        elif 'ftctr' in metrics_and_args['args']['expname']:
            metrics_and_args['method'] = 'SCaRF, finetuned (ours)'
        else:
            metrics_and_args['method'] = 'NeRF'        

        # add columns for arguments
        for k, v in metrics_and_args["args"].items():
            metrics_and_args[f"args.{k}"] = v
        del metrics_and_args["args"]

        if metrics_and_args['args.config'] in config_blacklist:
            print('Skipping', metrics_and_args['args.config'], '(blacklist)')
            continue

#         if 'fullres' in metrics_and_args['args.expname']:
#             print(metrics_and_args)
        assert metrics_and_args['args.half_res'] == True
    
        metrics_list.append(metrics_and_args)

metrics_df = pd.DataFrame(metrics_list)

Skipping configs/257_blender_materials_8views_ctr.txt (blacklist)
Skipping configs/257_blender_materials_8views_ctr.txt (blacklist)


# Group by scene, for all iterations

In [7]:
for scene_key, scene_df in metrics_df.groupby('args.datadir'):
    print(scene_key)
    scene_df = scene_df.sort_values([
        'views',
        'train_iters'
    ])
    display(scene_df)
#     print('==='*20)

./data/nerf_synthetic/chair


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
30,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,0.017929,18.436800,0.799504,0.190794,199999,4,SCaRF (ours),configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,0.004178,24.620086,0.897281,0.094988,159999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.004170,24.633213,0.898032,0.092438,199999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
48,0.004170,24.633213,0.898032,0.092438,199999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
50,0.011285,21.048566,0.860601,0.162509,199999,8,NeRF,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.000412,34.073013,0.977810,0.024821,199999,100,NeRF,configs/chair.txt,blender_paper_chair,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/drums


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
6,0.045604,13.457292,0.675309,0.301415,199999,4,SCaRF (ours),configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,0.009931,20.154216,0.845756,0.115702,159999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.018335,17.471725,0.769530,0.231024,199999,8,NeRF,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.010230,20.033635,0.843434,0.117413,199999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
38,0.010230,20.033635,0.843434,0.117413,199999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
54,0.002971,25.529593,0.928950,0.063527,199999,100,NeRF,configs/drums.txt,blender_paper_drums,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ficus


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
32,0.198314,7.085420,0.410553,0.524728,29999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,0.210504,6.808063,0.394229,0.532183,199999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
51,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,0.02,relu,1,False,200000.0,1.0,0.0,0.0,single_random,1.0
4,0.007130,21.699216,0.847500,0.112648,29999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,0.164257,7.869122,0.483181,0.470464,79999,8,NeRF,configs/237_blender_materials_8views.txt,237_blender_materials_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.008147,21.253789,0.842860,0.102170,199999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
12,0.001220,29.339644,0.958222,0.037147,199999,100,NeRF,configs/materials.txt,materials,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/mic


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
10,0.012596,19.239916,0.877721,0.120375,199999,4,SCaRF (ours),configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.002806,26.277187,0.943431,0.054970,149999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.002838,26.321344,0.943849,0.052638,199999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
15,0.003241,26.287436,0.944459,0.067203,199999,8,NeRF,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,0.002838,26.321344,0.943849,0.052638,199999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
11,0.000494,33.196878,0.978615,0.022581,199999,100,NeRF,configs/mic.txt,blender_paper_mic,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ship


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
21,0.017642,17.692778,0.676209,0.290592,199999,4,SCaRF (ours),configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,0.005805,22.835220,0.753288,0.205465,179999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.005507,23.043154,0.758032,0.204418,199999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
26,0.005507,23.043154,0.758032,0.204418,199999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
34,0.097770,12.929134,0.605394,0.375103,199999,8,NeRF,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,0.001215,29.407472,0.874514,0.124558,199999,100,NeRF,configs/ship.txt,blender_paper_ship,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Group by scene, for last iteration

In [8]:
# Select evaluations of most recent training iteration
idx = metrics_df.groupby('args.config')['train_iters'].transform(max) == metrics_df['train_iters']
metrics_df_max_iters = metrics_df[idx]

In [12]:
# Print cleanly by dataset
for scene_key, scene_df in metrics_df_max_iters.groupby('args.datadir'):
    print(scene_key)
    scene_df = scene_df.sort_values([
        'views',
        'train_iters',
        'method'
    ])
    display(scene_df)

./data/nerf_synthetic/chair


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
30,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,0.017929,18.436800,0.799504,0.190794,199999,4,SCaRF (ours),configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,0.011285,21.048566,0.860601,0.162509,199999,8,NeRF,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.004170,24.633213,0.898032,0.092438,199999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
48,0.004170,24.633213,0.898032,0.092438,199999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
9,0.000412,34.073013,0.977810,0.024821,199999,100,NeRF,configs/chair.txt,blender_paper_chair,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/drums


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
19,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.045604,13.457292,0.675309,0.301415,199999,4,SCaRF (ours),configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.018335,17.471725,0.769530,0.231024,199999,8,NeRF,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.010230,20.033635,0.843434,0.117413,199999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
38,0.010230,20.033635,0.843434,0.117413,199999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
54,0.002971,25.529593,0.928950,0.063527,199999,100,NeRF,configs/drums.txt,blender_paper_drums,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ficus


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
51,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,0.02,relu,1,False,200000.0,1.0,0.0,0.0,single_random,1.0
31,0.210504,6.808063,0.394229,0.532183,199999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
13,0.164257,7.869122,0.483181,0.470464,79999,8,NeRF,configs/237_blender_materials_8views.txt,237_blender_materials_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.008147,21.253789,0.842860,0.102170,199999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
12,0.001220,29.339644,0.958222,0.037147,199999,100,NeRF,configs/materials.txt,materials,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/mic


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
49,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0.012596,19.239916,0.877721,0.120375,199999,4,SCaRF (ours),configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.003241,26.287436,0.944459,0.067203,199999,8,NeRF,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.002838,26.321344,0.943849,0.052638,199999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
52,0.002838,26.321344,0.943849,0.052638,199999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
11,0.000494,33.196878,0.978615,0.022581,199999,100,NeRF,configs/mic.txt,blender_paper_mic,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./data/nerf_synthetic/ship


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
42,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.017642,17.692778,0.676209,0.290592,199999,4,SCaRF (ours),configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,0.097770,12.929134,0.605394,0.375103,199999,8,NeRF,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0.005507,23.043154,0.758032,0.204418,199999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
1,0.005507,23.043154,0.758032,0.204418,199999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
14,0.001215,29.407472,0.874514,0.124558,199999,100,NeRF,configs/ship.txt,blender_paper_ship,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Group by method, for last iteration, compute mean metrics

In [10]:
# Aggregate by method
grouped_df = metrics_df_max_iters\
    .sort_values(['args.max_train_views', 'args.expname'])\
    .groupby(['views', 'method'])

for k, v in grouped_df:
#     print(k)
    mse = v['mse'].mean()
    psnr = v['psnr'].mean()
    ssim = v['ssim'].mean()
    lpips = v['lpips'].mean()
    print(f'{k[0]} views, {k[1]} mse={mse:.4f} psnr={psnr:.4f} ssim={ssim:.4f} lpips={lpips:.4f}')
    if len(v) != 8:
        print('!!WARN bad number of scenes')
    display(v)
    print('='*50)

4 views, NeRF mse=0.1076 psnr=10.8934 ssim=0.5841 lpips=0.4143


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
30,0.052804,12.973991,0.601996,0.415411,199999,4,NeRF,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.086626,10.671803,0.590800,0.423424,199999,4,NeRF,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.018838,17.700611,0.827852,0.162645,199999,4,NeRF,configs/222_blender_ficus_4views.txt,222_blender_ficus_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,0.132863,8.943158,0.491976,0.475585,199999,4,NeRF,configs/223_blender_lego_4views.txt,223_blender_lego_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,0.058747,12.372348,0.707625,0.346917,199999,4,NeRF,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,0.241816,6.332760,0.335781,0.593170,199999,4,NeRF,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,0.103696,10.173140,0.619469,0.428852,199999,4,NeRF,configs/226_blender_hotdog_4views.txt,226_blender_hotdog_4views,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,0.165159,7.979751,0.497654,0.468705,199999,4,NeRF,configs/227_blender_materials_4views.txt,226_blender_materials_4views,./logs,...,0.02,relu,1,False,200000.0,1.0,0.0,0.0,single_random,1.0


4 views, SCaRF (ours) mse=0.0835 psnr=13.2029 ssim=0.6352 lpips=0.3397


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
47,0.017929,18.436800,0.799504,0.190794,199999,4,SCaRF (ours),configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.045604,13.457292,0.675309,0.301415,199999,4,SCaRF (ours),configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,0.074905,11.403763,0.638804,0.319521,199999,4,SCaRF (ours),configs/242_blender_ficus_4views_ctr.txt,242_blender_ficus_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,0.071053,11.619445,0.621743,0.360053,199999,4,SCaRF (ours),configs/243_blender_lego_4views_ctr.txt,243_blender_lego_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,0.012596,19.239916,0.877721,0.120375,199999,4,SCaRF (ours),configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,0.017642,17.692778,0.676209,0.290592,199999,4,SCaRF (ours),configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,0.217884,6.964789,0.398078,0.602763,199999,4,SCaRF (ours),configs/246_blender_hotdog_4views_ctr.txt,246_blender_hotdog_4views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
31,0.210504,6.808063,0.394229,0.532183,199999,4,SCaRF (ours),configs/247_blender_materials_4views_ctr.txt,247_blender_materials_4views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0


8 views, NeRF mse=0.0683 psnr=14.9531 ssim=0.6885 lpips=0.3179


...

,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
18,0.005002,23.896500,0.862688,0.110295,199999,8,SCaRF (ours),configs/042_blender_paper_lego.txt,042_blender_paper_lego_ctr_coarseandfine_clip_...,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.004170,24.633213,0.898032,0.092438,199999,8,SCaRF (ours),configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
16,0.010230,20.033635,0.843434,0.117413,199999,8,SCaRF (ours),configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
44,0.008987,20.744322,0.871760,0.097055,199999,8,SCaRF (ours),configs/252_blender_ficus_8views_ctr.txt,252_blender_ficus_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
7,0.002838,26.321344,0.943849,0.052638,199999,8,SCaRF (ours),configs/254_blender_mic_8views_ctr.txt,254_blender_mic_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
26,0.005507,23.043154,0.758032,0.204418,199999,8,SCaRF (ours),configs/255_blender_ship_8views_ctr.txt,255_blender_ship_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
39,0.004214,25.335648,0.907776,0.089148,189999,8,SCaRF (ours),configs/256_blender_hotdog_8views_ctr.txt,256_blender_hotdog_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0
5,0.008147,21.253789,0.842860,0.102170,199999,8,SCaRF (ours),configs/258_blender_materials_8views_ctr.txt,258_blender_materials_8views_ctr,./logs,...,0.02,relu,1,False,200000.0,10.0,0.0,0.0,single_random,1.0


8 views, SCaRF, finetuned (ours) mse=0.0061 psnr=23.1120 ssim=0.8630 lpips=0.1124
!!WARN bad number of scenes


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
52,0.002838,26.321344,0.943849,0.052638,199999,8,"SCaRF, finetuned (ours)",configs/282_blender_mic_8views_ftctr.txt,282_blender_mic_8views_ftctr254,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
48,0.004170,24.633213,0.898032,0.092438,199999,8,"SCaRF, finetuned (ours)",configs/283_blender_chair_8views_ftctr.txt,283_blender_chair_8views_ftctr250,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
38,0.010230,20.033635,0.843434,0.117413,199999,8,"SCaRF, finetuned (ours)",configs/284_blender_drums_8views_ftctr.txt,284_blender_drums_8views_ftctr251,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
35,0.008987,20.744322,0.871760,0.097055,199999,8,"SCaRF, finetuned (ours)",configs/285_blender_ficus_8views_ftctr.txt,285_blender_ficus_8views_ftctr252,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
25,0.005002,23.896500,0.862688,0.110295,199999,8,"SCaRF, finetuned (ours)",configs/286_blender_lego_8views_ftctr.txt,286_blender_lego_8views_ftctr042,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0
1,0.005507,23.043154,0.758032,0.204418,199999,8,"SCaRF, finetuned (ours)",configs/289_blender_ship_8views_ftctr.txt,289_blender_ship_8views_ftctr255,./logs,...,0.02,relu,1,False,400000.0,1.0,0.0,0.0,single_random,1.0


100 views, NeRF mse=0.0011 psnr=31.1532 ssim=0.9538 lpips=0.0456


,mse,psnr,ssim,lpips,train_iters,views,method,args.config,args.expname,args.basedir,...,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus,args.use_softplus_alpha,args.N_iters,args.i_log_ctr_img,args.render_increase_interval_every,args.render_increase_interval_delta,args.consistency_loss_sampling,args.consistency_loss_sampling_temp
9,0.000412,34.073013,0.977810,0.024821,199999,100,NeRF,configs/chair.txt,blender_paper_chair,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,0.002971,25.529593,0.928950,0.063527,199999,100,NeRF,configs/drums.txt,blender_paper_drums,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,0.001242,29.162503,0.966199,0.035262,199999,100,NeRF,configs/ficus.txt,blender_paper_ficus,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,0.000721,31.617750,0.965375,0.032724,199999,100,NeRF,configs/lego.txt,blender_paper_lego,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,0.000494,33.196878,0.978615,0.022581,199999,100,NeRF,configs/mic.txt,blender_paper_mic,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,0.001215,29.407472,0.874514,0.124558,199999,100,NeRF,configs/ship.txt,blender_paper_ship,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,0.000254,36.898710,0.980554,0.024509,199999,100,NeRF,configs/hotdog.txt,hotdog,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,0.001220,29.339644,0.958222,0.037147,199999,100,NeRF,configs/materials.txt,materials,./logs,...,0.02,relu,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
!ls /data/ajay/clip/nerf/nerf-pytorch/data/nerf_synthetic/chair/train -1 | wc -l

100
